<a href="https://colab.research.google.com/github/xingguangYan/Landsat-5-NDWI-image-restoration/blob/main/%E2%80%9C%E5%BD%B1%E5%83%8F%E5%88%86%E5%89%B2_%E7%9F%A2%E9%87%8F%E5%AF%BC%E5%87%BA_%E5%9D%90%E6%A0%87%E8%8E%B7%E5%8F%96_%E9%9D%A2%E7%A7%AF%E7%AD%9B%E9%80%89__%E9%A3%9E%E6%9C%BA%E5%9C%BA%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/135_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Earth Engine Image Segmentation with the Segment Anything Model**

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
 !pip install -U geemap

In [ ]:
 %pip install segment-geospatial pycrs

In [ ]:
%pip install leafmap

In [17]:
import ee
import geemap
import leafmap
from samgeo import SamGeo
from samgeo import SamGeo, show_image, download_file, overlay_images, tms_to_geotiff

In [18]:
# 这个是北京机场的位置
m =  geemap.Map(center=[40.08273107546195,116.60838761693236 ], zoom=18)
m.add_basemap("SATELLITE")
m

Map(center=[40.08273107546195, 116.60838761693236], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
table = ee.FeatureCollection("projects/ee-dalunwen/assets/HDmeitanbianjie")
#将属性插入点
#创建特征长度列表
listaIDs = ee.List.sequence(0, table.size().subtract(1))
#print('listaIDs',listaIDs)

# 在列表中转换特征集合
lista = table.toList(table.size())
#print('lista',lista)

In [ ]:
#创建属性
def ccc(novoIndice):
  #选择列表元素
  feature = ee.Feature(lista.get(novoIndice))
  # 提取每个特征的几何形状
  geometry = feature.geometry()
  area = feature.area(0.1)
  # 提取坐标
  coords = ee.List(geometry.centroid().coordinates())
  lon = coords.get(0)
  lat = coords.get(1)
  # 将数字格式化为字符串
  indiceFormatado = ee.Number(novoIndice).format('%1d')

  # 给每个矢量添加坐标信息
  return feature.set(
      "ID",ee.Number.parse(indiceFormatado),
      "Longitude",lon,
      "Latitude",lat,
      "Area",area
       )


In [ ]:
assetID = ee.FeatureCollection(listaIDs.map(ccc))
print('Novos Pontos Aleatórios',assetID.getInfo())


Novos Pontos Aleatórios {'type': 'FeatureCollection', 'columns': {'Area': 'Float', 'ID': 'Integer', 'Id': 'Integer', 'Latitude': 'Float', 'Longitude': 'Float', 'name': 'String', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[112.20677661673449, 36.76685334599265], [112.21597574892438, 36.76288474329051], [112.21584649402595, 36.75450610053507], [112.22283388649882, 36.754434761547124], [112.22239245667096, 36.726502925768344], [112.24594981721263, 36.726262137172114], [112.24627090258652, 36.746350388097554], [112.25460045558498, 36.74626123568922], [112.25486359878704, 36.762478986881675], [112.22804653450342, 36.76275985716416], [112.22830067652957, 36.7787056312552], [112.21351881281754, 36.77885722967641], [112.21344749967872, 36.77435356321087], [112.20689704903621, 36.77442044180599], [112.20677661673449, 36.76685334599265]]]}, 'id': '00000000000000000000', 'properties': {'Area': 13294885.185948262, 'ID': 0, 'Id': 0, 

In [ ]:

#可视化
m.addLayer(assetID,{palette:'black'}, 'Pontos Amostrais')

Afilter = assetID.filter(ee.Filter.lt(**{
    'name':'Area',
    'value':30,
})).filter(ee.Filter.gt(**{'name':'Area','value':7}))

m.addLayer(Afilter,vis_params={},name='Afilter')
m

NameError: name 'palette' is not defined

In [4]:
#m =  geemap.Map(center=[40.08273107546195,116.60838761693236 ], zoom=18)
m =  leafmap.Map(center=[40.08273107546195,116.60838761693236], zoom=19)
m.add_basemap("SATELLITE")
m

Map(center=[40.08273107546195, 116.60838761693236], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [5]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [116.60265340885371, 40.08408352796448 , 116.60458728155345, 40.082690097786354]

In [ ]:
'''    ee.Geometry.Polygon(
        [[[116.59990146240443, 40.08421486600977],
          [116.59990146240443, 40.080061177628956],
          [116.6105766542837, 40.080061177628956],
          [116.6105766542837, 40.08421486600977]]], null, false);

              ee.Geometry.Polygon(
        [[[116.60265340885371, 40.08408352796448],
          [116.60265340885371, 40.082690097786354],
          [116.60458728155345, 40.082690097786354],
          [116.60458728155345, 40.08408352796448]]]
'''

In [6]:
print(bbox)

[116.60265340885371, 40.08408352796448, 116.60458728155345, 40.082690097786354]


In [7]:
image = "satellitexiao.tif"

In [8]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 01/42
Downloaded image 02/42
Downloaded image 03/42
Downloaded image 04/42
Downloaded image 05/42
Downloaded image 06/42
Downloaded image 07/42
Downloaded image 08/42
Downloaded image 09/42
Downloaded image 10/42
Downloaded image 11/42
Downloaded image 12/42
Downloaded image 13/42
Downloaded image 14/42
Downloaded image 15/42
Downloaded image 16/42
Downloaded image 17/42
Downloaded image 18/42
Downloaded image 19/42
Downloaded image 20/42
Downloaded image 21/42
Downloaded image 22/42
Downloaded image 23/42
Downloaded image 24/42
Downloaded image 25/42
Downloaded image 26/42
Downloaded image 27/42
Downloaded image 28/42
Downloaded image 29/42
Downloaded image 30/42
Downloaded image 31/42
Downloaded image 32/42
Downloaded image 33/42
Downloaded image 34/42
Downloaded image 35/42
Downloaded image 36/42
Downloaded image 37/42
Downloaded image 38/42
Downloaded image 39/42
Downloaded image 40/42
Downloaded image 41/42
Downloaded image 42/42
Saving GeoTIFF. Please wait...
Ima

In [21]:

m.layers[-1].visible = False  # 关闭原有的卫星底图
m.add_raster(image, layer_name="Image") #加载你的影像在地图上
m

Map(bottom=50772046.0, center=[40.08273107546195, 116.60838761693236], controls=(ZoomControl(options=['positio…

In [ ]:
m.layers[-1].visible = False  # 关闭原有的卫星底图
m.add_raster(image, layer_name="Image") #加载你的影像在地图上
m

NameError: name 'm' is not defined

In [9]:
sam = SamGeo(
    model_type="vit_h",  # can be vit_h, vit_b, vit_l, vit_tiny
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

In [10]:
mask = "segmentxiao.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

100%|██████████| 12/12 [1:54:36<00:00, 573.00s/it]


In [11]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)
sam.tiff_to_shp

<bound method SamGeo.tiff_to_shp of <samgeo.samgeo.SamGeo object at 0x792868036860>>

In [26]:
sss = ee.FeatureCollection(segment)
print(sss.getInfo())

NameError: name 'segment' is not defined

In [24]:
geemap.ee_export_vector_to_asset(collection=sss,description='myExportTableTasksegmentxiao', assetId='users_bqt2000204051', maxVertices=None)

users/bqt2000204051/users_bqt2000204051
Exporting myExportTableTasksegmentxiao... Please check the Task Manager from the JavaScript Code Editor.


EEException: Collection.loadTable: Collection asset 'segment.shp' not found.

geemap中代码的转换
这里我们将我们所用到的矢量中心点的获取工作进行转换，并按照一定的面积来筛除面积较大的区域，因为我们要获取电线杆的中心位置，所以这里我们基本上就需要按照面积来筛选。这里有一个问题就是我们导出的是shp文件，所以不知道这里是否可以直接使用ee.FeatureCollection（）将其将转化为矢量集合进行计算。

In [ ]:
javascript_code = """
var table = ee.FeatureCollection("projects/ee-dalunwen/assets/HDmeitanbianjie");
//将属性插入点
//创建特征长度列表
var listaIDs = ee.List.sequence(0, table.size().subtract(1));
print('listaIDs',listaIDs)
// 在列表中转换特征集合
var lista = table.toList(table.size());
print('lista',lista)
//创建属性
var assetID = ee.FeatureCollection(listaIDs.map(function(novoIndice) {
  //选择列表元素
  var feature = ee.Feature(lista.get(novoIndice));

  // 提取每个特征的几何形状
  var geometry = feature.geometry();
  var area = feature.area(0.1);
  // 提取坐标
  var coords = ee.List(geometry.centroid().coordinates());
  var lon = coords.get(0);
  var lat = coords.get(1);

  // 将数字格式化为字符串
  var indiceFormatado = ee.Number(novoIndice).format('%1d');

  // 给每个矢量添加坐标信息
  return feature.set({
    // 'system:index': indiceFormatado,
    'ID': ee.Number.parse(indiceFormatado),
    'Longitude': lon,
    'Latitude': lat,
    'Area':area
  });
}));

print('Novos Pontos Aleatórios',assetID)

//可视化
Map.addLayer(assetID,{}, 'Pontos Amostrais');


var Afilter = assetID.filter(ee.Filter.lt({
		name:'Area',
		value:5000000,
}))
print(Afilter)


"""

In [ ]:
lines = geemap.js_snippet_to_py(
    javascript_code, add_new_cell=False,
    import_ee=True, import_geemap=True, show_map=True)
for line in lines:
    print(line.rstrip())


import ee
import geemap

m = geemap.Map()

table = ee.FeatureCollection("projects/ee-dalunwen/assets/HDmeitanbianjie")
#将属性插入点
#创建特征长度列表
listaIDs = ee.List.sequence(0, table.size().subtract(1))
print('listaIDs',listaIDs)
# 在列表中转换特征集合
lista = table.toList(table.size())
print('lista',lista)
#创建属性

def func_fvv(novoIndice):
  #选择列表元素
  feature = ee.Feature(lista.get(novoIndice))

  # 提取每个特征的几何形状
  geometry = feature.geometry()
  area = feature.area(0.1)
  # 提取坐标
  coords = ee.List(geometry.centroid().coordinates())
  lon = coords.get(0)
  lat = coords.get(1)

  # 将数字格式化为字符串
  indiceFormatado = ee.Number(novoIndice).format('%1d')

  # 给每个矢量添加坐标信息
  return feature.set(**{
    # 'system:index': indiceFormatado,
    'ID': ee.Number.parse(indiceFormatado),
    'Longitude': lon,
    'Latitude': lat,
    'Area':area
  })

assetID = ee.FeatureCollection(listaIDs.map(func_fvv
))

))

print('Novos Pontos Aleatórios',assetID)

#可视化
m.addLayer(assetID,{}, 'Pontos Amostrais')

Afilter = assetID.filter

In [ ]:
table = ee.FeatureCollection("projects/ee-dalunwen/assets/HDmeitanbianjie")
#将属性插入点
#创建特征长度列表
listaIDs = ee.List.sequence(0, table.size().subtract(1))
print('listaIDs',listaIDs)
# 在列表中转换特征集合
lista = table.toList(table.size())
print('lista',lista)
#创建属性

def func_fvv(novoIndice):
  #选择列表元素
  feature = ee.Feature(lista.get(novoIndice))

  # 提取每个特征的几何形状
  geometry = feature.geometry()
  area = feature.area(0.1)
  # 提取坐标
  coords = ee.List(geometry.centroid().coordinates())
  lon = coords.get(0)
  lat = coords.get(1)

  # 将数字格式化为字符串
  indiceFormatado = ee.Number(novoIndice).format('%1d')

  # 给每个矢量添加坐标信息
  return feature.set({
    # 'system:index': indiceFormatado,
    'ID': ee.Number.parse(indiceFormatado),
    'Longitude': lon,
    'Latitude': lat,
    'Area':area
  })

assetID = ee.FeatureCollection(listaIDs.map(func_fvv))


print('Novos Pontos Aleatórios',assetID.getInfo())




listaIDs ee.List({
  "functionInvocationValue": {
    "functionName": "List.sequence",
    "arguments": {
      "end": {
        "functionInvocationValue": {
          "functionName": "Number.subtract",
          "arguments": {
            "left": {
              "functionInvocationValue": {
                "functionName": "Collection.size",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.loadTable",
                      "arguments": {
                        "tableId": {
                          "constantValue": "projects/ee-dalunwen/assets/HDmeitanbianjie"
                        }
                      }
                    }
                  }
                }
              }
            },
            "right": {
              "constantValue": 1
            }
          }
        }
      },
      "start": {
        "constantValue": 0
      }
    }
  }
})
lista ee.L

# 新段落

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point(-122.259679, 37.871838)
collection = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(point)
    .filterDate("2008-01-01", "2018-01-01")
    .filter(ee.Filter.listContains("system:band_names", "N"))
)
image = collection.first()
Map.addLayer(image, {}, "NAIP")
Map.centerObject(point, 16)
Map

In [ ]:
bbox = Map.user_roi_coords()
if bbox is None:
    bbox = [-122.2666, 37.8682, -122.252, 37.8752]

In [ ]:
geemap.ee_to_geotiff(
    image, "naip.tif", bbox, zoom=17, vis_params={"bands": ["R", "G", "B"]}
)

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    device=None,
    sam_kwargs=None,
)

In [ ]:
sam.generate("naip.tif", output="masks.tif", foreground=True, unique=True)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")

In [ ]:
Map.add_raster("annotations.tif", opacity=0.5, layer_name="Masks")
Map

In [ ]:
sam.tiff_to_vector("masks.tif", "masks.shp")

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
Map.add_vector("masks.shp", layer_name="Vector", style=style)
Map